In [60]:
import numpy as np
import matplotlib.pyplot as plt
import os, argparse, math, random
from datetime import datetime

testFiles = []
ka_test_file = "../dataset/prep/codebook/test_c0_"
kAA_test_file = "../dataset/prep/codebook/test_c1_"
kha_test_file = "../dataset/prep/codebook/test_c2_"

ka_train_file = "../dataset/prep/codebook/train_c0_"
kAA_train_file = "../dataset/prep/codebook/train_c1_"
kha_train_file = "../dataset/prep/codebook/train_c2_"

fileName = "../dataset/prep/codebook/train_c"

outputPath = "../output/abpi/NEW"

# for every m and n combination (m = 2,4,8,16,32) and (n = 2,3,4,5)

# N = [2,3,4,5]
# M = [4,8,16,32]

N = [2,3,4,5]
M = [16]

wholeData = []

A = []
B = []
PI = []

In [61]:
def fileHandle(fileName):
    global wholeData
    wholeData = []
    file = open(fileName)
    for line in file:
        teLine = line.rstrip('\n ').split(' ')
        nLine = [int(i) for i in teLine]
        nLine = np.array(nLine)
#         print(nLine)
        wholeData.append(nLine)
    file.close()

def writeFile(classIndex, name, values, n, m, ext):
    targetPath = outputPath + name + "_c" + str(classIndex) + "_n" + str(n) + "_m" + str(m) + ext

    if not os.path.exists(os.path.dirname(targetPath)):
        try:
            os.makedirs(os.path.dirname(targetPath))
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    try:
        print("target File: ", targetPath)
        outfile = open(targetPath, "w")
    except IOError:
        print("File not created !!!!!!!!!!!!!!!!!!!!!!!!!")

    for value in values:
        for feature in value:
            outfile.write(str(feature)+" ")
        outfile.write("\n")
    outfile.close()
    print("Done !")

In [62]:
def EMAlgo(n, m):
    global A
    global B
    global PI
    global wholeData
    
    probI = 1e9
    probF = 0
    
    iterCount = 0
    while (abs(probI - probF) >= 1e-3) and (iterCount < 169):
        print("Iteration Number: ", iterCount, " ; probI: ",probI, " ; probF: ", probF, "xxxx: ",abs(probI - probF))
        iterCount += 1
        newA = np.zeros((n, n))
        newB = np.zeros((n, m))
        newPI = np.zeros((n))
        
#         print(B)
        
        # remind to send old - A, B in functions(forward, backward)
        # and calc of zeta, gamma
        # also remember to equate A = newA at the end of following for-loop!!
        
        probI = probF
        probF = 0
                
        for obs in wholeData:
            
            tempPLog = 0
            # From here, E-step start!!
            # See for alpha, beta to be set
            alpha = forwdProc(obs, n)
            beta = backwdProc(obs, n)
            
            lObs = len(obs)
            
            for i in range(n):
                tempPLog += alpha[lObs-1,i]
            
            zeta = np.zeros((lObs-1, n, n))
                
            # pass the n as parameter in the func EMAlgo itself
            
            # common denominator
            deno = np.zeros((lObs-1))
            
            for ti in range(lObs-1):
                for i in range(n):
                    for j in range(n):
                        deno[ti] += alpha[ti,i] * A[i,j] * B[j,obs[ti+1]] * beta[ti+1,j]
                        
            
            for ti in range(lObs-1):
                for i in range(n):
                    for j in range(n):
                        zeta[ti,i,j] = alpha[ti,i] * A[i,j] * B[j,obs[ti+1]] * beta[ti+1,j]
                        zeta[ti,i,j] /= deno[ti]
                        
            
            
            gamma = np.zeros((lObs, n))
            deno = np.zeros((lObs))
            
            for ti in range(lObs):
                for i in range(n):
                    deno[ti] += alpha[ti,i] * beta[ti,i]
                    
            
            for ti in range(lObs):
                for i in range(n):
                    gamma[ti,i] = alpha[ti,i] * beta[ti,i]
                    gamma[ti,i] /= deno[ti]
                    
            
            # E-step ends
            
            # M-step begins
            
            a = np.zeros((n, n))
            b = np.zeros((n, m))
            pi = np.zeros((n))
            
            for i in range(n):
                pi[i] = gamma[0,i]
                
            
            for i in range(n):
                denoGamma = 0
                
                for ti in range(lObs-1):
                    denoGamma += gamma[ti,i]
                
                for j in range(n):
                    nrtZeta = 0
                    for ti in range(lObs-1):
                        nrtZeta += zeta[ti,i,j]
                    
                    a[i,j] = nrtZeta / denoGamma
                    
            
            # loop over the observation sequence!!

            for ti,syb in enumerate(obs):

                for j in range(n):

                    b[j,int(syb)] += gamma[ti,j]


            for j in range(n):
                tempVal = 0
                for ti in range(lObs):
                    tempVal += gamma[ti,j]

                for k in range(m):
                    b[j,k] /= tempVal


            newA += a
            newB += b
            newPI += pi
            

            probF += np.log(tempPLog)
            
        lData = len(wholeData)
        
        A = newA / lData
        B = newB / lData
        PI = newPI / lData
    

In [63]:
def initialize(n, m, classIndex):
    # call to EM from here!!
    # for each class loop
    # read the file from here itself for every class separately loop on the class itself 
    # outer loop for the class
    
    global A
    global B
    global PI
    global wholeData
    
    #treat them as global!!
    A = np.zeros((n,n))
    B = np.zeros((n,m))
    PI = np.zeros((n))
    
    for obs in wholeData:
        # obs is my numpy array
        
        a = np.zeros((n,n))
        b = np.zeros((n,m))
        
        lgt = len(obs)
        gone = int(lgt / n)
        toFirst = lgt - gone * n
        toFirst += gone
                
        for i in range(n-1):
            if i == 0:
                a[i,i] = (toFirst - 1) / toFirst
                a[i,i+1] = 1 / toFirst
            elif gone != 0:
                a[i,i] = (gone - 1) / gone
                a[i,i+1] = 1 / gone
        
        if gone != 0:
            a[n-1][n-1] = 1
            
        j = 0
        anth = 0
        for i,syb in enumerate(obs):
            if j == 0 and i < toFirst:
                b[0,int(syb)] += 1
                anth += 1
                continue
            elif j == 0:
                j = 1
                anth = 0
                
            if anth >= gone:
                j += 1
                anth = 0
            
            b[j,int(syb)] += 1
            anth += 1
            
        for i in range(n):
            if i == 0:
                b[i] /= toFirst
            elif gone != 0:
                b[i] /= gone
                
        A += a
        B += b
        
        
    lData = len(wholeData)
    A /= lData
    B /= lData
    PI[0] = 1
    
    #return from EMAlgo A, B, PI
    EMAlgo(n, m)    
    
    writeFile(classIndex, "em", A, n, m, ".a")
    writeFile(classIndex, "em", B, n, m, ".b")
    writeFile(classIndex, "em", [PI], n, m, ".pi")
    

In [64]:
def forwdProc(obs, n):
    
    global A
    global B
    global PI
    global wholeData
    
    # Assuming I know n
    lObs = len(obs)
    
    alpha = np.zeros((lObs, n))
    for i in range(n):
        alpha[0,i] = PI[i] * B[i,obs[0]]
        
    for ti in range(1,lObs):
        for j in range(n):
            storeVal = 0
            for i in range(n):
                storeVal += alpha[ti-1,i] * A[i,j]
            alpha[ti,j] = storeVal * B[j, obs[ti]]
    
    return alpha

In [65]:
def backwdProc(obs, n):
    # assuming I know n
    
    global A
    global B
    global PI
    global wholeData
    
    lObs = len(obs)
    beta = np.zeros((lObs, n))
    
    for i in range(n):
        beta[lObs-1,i] = 1
    
    for ti in range(lObs-2, -1, -1):
        for i in range(n):
            storeVal = 0
            for j in range(n):
                storeVal += A[i,j] * B[j,obs[ti+1]] * beta[ti+1,j]
            beta[ti,i] = storeVal
    
    return beta

In [66]:
for n in N:
    for m in M:
        # now read the files and store the data in wholeData
        # list of numpy arrays - wholeData[0] = one observation sequence, one numpy array
        print("M: ", m)
        for i in range(3):
            fileHandle(fileName+str(i)+"_"+ str(m) + ".cb")
            print("filename: ", (fileName+str(i)+"_"+ str(m) + ".cb") ,  "  ; data size: ",len(wholeData))
            initialize(n,m,i);

M:  16
filename:  ../dataset/prep/codebook/train_c0_16.cb   ; data size:  383
Iteration Number:  0  ; probI:  1000000000.0  ; probF:  0 xxxx:  1000000000.0
Iteration Number:  1  ; probI:  0  ; probF:  -19964.17028599451 xxxx:  19964.17028599451
Iteration Number:  2  ; probI:  -19964.17028599451  ; probF:  -19511.980371349844 xxxx:  452.1899146446667
Iteration Number:  3  ; probI:  -19511.980371349844  ; probF:  -19428.554052815394 xxxx:  83.42631853444982
Iteration Number:  4  ; probI:  -19428.554052815394  ; probF:  -19412.447958196637 xxxx:  16.106094618757197
Iteration Number:  5  ; probI:  -19412.447958196637  ; probF:  -19408.106582769 xxxx:  4.341375427637104
Iteration Number:  6  ; probI:  -19408.106582769  ; probF:  -19406.888375607716 xxxx:  1.2182071612842265
Iteration Number:  7  ; probI:  -19406.888375607716  ; probF:  -19406.42383861772 xxxx:  0.46453698999539483
Iteration Number:  8  ; probI:  -19406.42383861772  ; probF:  -19406.283876807625 xxxx:  0.13996181009497377
It

Iteration Number:  13  ; probI:  -18461.410987228643  ; probF:  -18459.354028569363 xxxx:  2.0569586592791893
Iteration Number:  14  ; probI:  -18459.354028569363  ; probF:  -18457.79094361902 xxxx:  1.5630849503431818
Iteration Number:  15  ; probI:  -18457.79094361902  ; probF:  -18456.608858423428 xxxx:  1.1820851955926628
Iteration Number:  16  ; probI:  -18456.608858423428  ; probF:  -18455.72099529078 xxxx:  0.8878631326479081
Iteration Number:  17  ; probI:  -18455.72099529078  ; probF:  -18455.05836970103 xxxx:  0.6626255897499504
Iteration Number:  18  ; probI:  -18455.05836970103  ; probF:  -18454.566208008393 xxxx:  0.4921616926367278
Iteration Number:  19  ; probI:  -18454.566208008393  ; probF:  -18454.201713928564 xxxx:  0.3644940798294556
Iteration Number:  20  ; probI:  -18454.201713928564  ; probF:  -18453.93206480485 xxxx:  0.2696491237147711
Iteration Number:  21  ; probI:  -18453.93206480485  ; probF:  -18453.732488971316 xxxx:  0.19957583353243535
Iteration Number:

Iteration Number:  14  ; probI:  -3658.706105444282  ; probF:  -3658.628572187542 xxxx:  0.07753325674002554
Iteration Number:  15  ; probI:  -3658.628572187542  ; probF:  -3658.5503961444642 xxxx:  0.07817604307774673
Iteration Number:  16  ; probI:  -3658.5503961444642  ; probF:  -3658.4741531803543 xxxx:  0.07624296410995157
Iteration Number:  17  ; probI:  -3658.4741531803543  ; probF:  -3658.401097877115 xxxx:  0.07305530323947096
Iteration Number:  18  ; probI:  -3658.401097877115  ; probF:  -3658.331815189415 xxxx:  0.06928268769979695
Iteration Number:  19  ; probI:  -3658.331815189415  ; probF:  -3658.266527245906 xxxx:  0.06528794350879252
Iteration Number:  20  ; probI:  -3658.266527245906  ; probF:  -3658.2052507285625 xxxx:  0.061276517343685555
Iteration Number:  21  ; probI:  -3658.2052507285625  ; probF:  -3658.1478845287975 xxxx:  0.05736619976505608
Iteration Number:  22  ; probI:  -3658.1478845287975  ; probF:  -3658.0942614788687 xxxx:  0.05362304992877398
Iteration

Iteration Number:  102  ; probI:  -3657.20984714909  ; probF:  -3657.208299735209 xxxx:  0.0015474138808713178
Iteration Number:  103  ; probI:  -3657.208299735209  ; probF:  -3657.2067959127244 xxxx:  0.0015038224846648518
Iteration Number:  104  ; probI:  -3657.2067959127244  ; probF:  -3657.2053342189283 xxxx:  0.001461693796045438
Iteration Number:  105  ; probI:  -3657.2053342189283  ; probF:  -3657.2039132512873 xxxx:  0.0014209676410246175
Iteration Number:  106  ; probI:  -3657.2039132512873  ; probF:  -3657.202531664439 xxxx:  0.0013815868483106897
Iteration Number:  107  ; probI:  -3657.202531664439  ; probF:  -3657.2011881673743 xxxx:  0.0013434970646812872
Iteration Number:  108  ; probI:  -3657.2011881673743  ; probF:  -3657.199881520763 xxxx:  0.0013066466112832131
Iteration Number:  109  ; probI:  -3657.199881520763  ; probF:  -3657.1986105344677 xxxx:  0.0012709862953670381
Iteration Number:  110  ; probI:  -3657.1986105344677  ; probF:  -3657.1973740651574 xxxx:  0.001

Iteration Number:  17  ; probI:  -31749.181379479458  ; probF:  -31751.15296754347 xxxx:  1.9715880640105752
Iteration Number:  18  ; probI:  -31751.15296754347  ; probF:  -31752.792464153925 xxxx:  1.6394966104562627
Iteration Number:  19  ; probI:  -31752.792464153925  ; probF:  -31754.212903207048 xxxx:  1.4204390531231184
Iteration Number:  20  ; probI:  -31754.212903207048  ; probF:  -31755.453156936823 xxxx:  1.2402537297748495
Iteration Number:  21  ; probI:  -31755.453156936823  ; probF:  -31756.495022853942 xxxx:  1.0418659171191393
Iteration Number:  22  ; probI:  -31756.495022853942  ; probF:  -31757.303603831064 xxxx:  0.8085809771218919
Iteration Number:  23  ; probI:  -31757.303603831064  ; probF:  -31757.835185249107 xxxx:  0.5315814180430607
Iteration Number:  24  ; probI:  -31757.835185249107  ; probF:  -31758.013072540452 xxxx:  0.1778872913455416
Iteration Number:  25  ; probI:  -31758.013072540452  ; probF:  -31757.68880146375 xxxx:  0.3242710767008248
Iteration Num

Iteration Number:  94  ; probI:  -31727.089999799045  ; probF:  -31719.9421841081 xxxx:  7.147815690947027
Iteration Number:  95  ; probI:  -31719.9421841081  ; probF:  -31714.053403351867 xxxx:  5.8887807562314265
Iteration Number:  96  ; probI:  -31714.053403351867  ; probF:  -31710.144563618313 xxxx:  3.908839733554487
Iteration Number:  97  ; probI:  -31710.144563618313  ; probF:  -31707.82355191251 xxxx:  2.321011705804267
Iteration Number:  98  ; probI:  -31707.82355191251  ; probF:  -31706.44043731086 xxxx:  1.3831146016491402
Iteration Number:  99  ; probI:  -31706.44043731086  ; probF:  -31705.553351413626 xxxx:  0.8870858972331916
Iteration Number:  100  ; probI:  -31705.553351413626  ; probF:  -31704.9360085354 xxxx:  0.617342878227646
Iteration Number:  101  ; probI:  -31704.9360085354  ; probF:  -31704.4767888364 xxxx:  0.45921969899791293
Iteration Number:  102  ; probI:  -31704.4767888364  ; probF:  -31704.112130729096 xxxx:  0.3646581073044217
Iteration Number:  103  ; 

Iteration Number:  30  ; probI:  -3616.031726856284  ; probF:  -3616.038763675787 xxxx:  0.007036819502900471
Iteration Number:  31  ; probI:  -3616.038763675787  ; probF:  -3616.0445845249606 xxxx:  0.005820849173687748
Iteration Number:  32  ; probI:  -3616.0445845249606  ; probF:  -3616.0493820106863 xxxx:  0.0047974857257031545
Iteration Number:  33  ; probI:  -3616.0493820106863  ; probF:  -3616.053322101978 xxxx:  0.0039400912919518305
Iteration Number:  34  ; probI:  -3616.053322101978  ; probF:  -3616.056546819931 xxxx:  0.0032247179528894776
Iteration Number:  35  ; probI:  -3616.056546819931  ; probF:  -3616.059176949277 xxxx:  0.0026301293460164743
Iteration Number:  36  ; probI:  -3616.059176949277  ; probF:  -3616.061314654599 xxxx:  0.0021377053217292996
Iteration Number:  37  ; probI:  -3616.061314654599  ; probF:  -3616.0630459287404 xxxx:  0.001731274141548056
Iteration Number:  38  ; probI:  -3616.0630459287404  ; probF:  -3616.0644428329283 xxxx:  0.00139690418791360

Iteration Number:  36  ; probI:  -31148.809697690715  ; probF:  -31147.1835158772 xxxx:  1.62618181351354
Iteration Number:  37  ; probI:  -31147.1835158772  ; probF:  -31145.785903858843 xxxx:  1.3976120183579042
Iteration Number:  38  ; probI:  -31145.785903858843  ; probF:  -31144.65281179461 xxxx:  1.1330920642321871
Iteration Number:  39  ; probI:  -31144.65281179461  ; probF:  -31143.761817271217 xxxx:  0.8909945233936014
Iteration Number:  40  ; probI:  -31143.761817271217  ; probF:  -31143.068409604963 xxxx:  0.6934076662546431
Iteration Number:  41  ; probI:  -31143.068409604963  ; probF:  -31142.52981157416 xxxx:  0.5385980308019498
Iteration Number:  42  ; probI:  -31142.52981157416  ; probF:  -31142.113889804736 xxxx:  0.4159217694250401
Iteration Number:  43  ; probI:  -31142.113889804736  ; probF:  -31141.798987205886 xxxx:  0.31490259884958505
Iteration Number:  44  ; probI:  -31141.798987205886  ; probF:  -31141.570591095777 xxxx:  0.2283961101093155
Iteration Number:  

Iteration Number:  111  ; probI:  -31103.590107487234  ; probF:  -31103.575726402643 xxxx:  0.014381084591150284
Iteration Number:  112  ; probI:  -31103.575726402643  ; probF:  -31103.564076832736 xxxx:  0.011649569907604018
Iteration Number:  113  ; probI:  -31103.564076832736  ; probF:  -31103.5546929382 xxxx:  0.009383894535858417
Iteration Number:  114  ; probI:  -31103.5546929382  ; probF:  -31103.547187344062 xxxx:  0.007505594137910521
Iteration Number:  115  ; probI:  -31103.547187344062  ; probF:  -31103.541237631467 xxxx:  0.005949712594883749
Iteration Number:  116  ; probI:  -31103.541237631467  ; probF:  -31103.536575267513 xxxx:  0.0046623639536846895
Iteration Number:  117  ; probI:  -31103.536575267513  ; probF:  -31103.53297649969 xxxx:  0.0035987678238598164
Iteration Number:  118  ; probI:  -31103.53297649969  ; probF:  -31103.530254842946 xxxx:  0.002721656743233325
Iteration Number:  119  ; probI:  -31103.530254842946  ; probF:  -31103.528254864537 xxxx:  0.001999

Iteration Number:  63  ; probI:  -3619.245837524521  ; probF:  -3619.243758850594 xxxx:  0.002078673926916963
Iteration Number:  64  ; probI:  -3619.243758850594  ; probF:  -3619.241836504335 xxxx:  0.0019223462591071439
Iteration Number:  65  ; probI:  -3619.241836504335  ; probF:  -3619.2400573813015 xxxx:  0.0017791230334296415
Iteration Number:  66  ; probI:  -3619.2400573813015  ; probF:  -3619.2384096007354 xxxx:  0.0016477805661452294
Iteration Number:  67  ; probI:  -3619.2384096007354  ; probF:  -3619.236882376013 xxxx:  0.0015272247223947488
Iteration Number:  68  ; probI:  -3619.236882376013  ; probF:  -3619.2354659009884 xxxx:  0.001416475024598185
Iteration Number:  69  ; probI:  -3619.2354659009884  ; probF:  -3619.234151249954 xxxx:  0.0013146510345904971
Iteration Number:  70  ; probI:  -3619.234151249954  ; probF:  -3619.2329302893236 xxxx:  0.001220960630234913
Iteration Number:  71  ; probI:  -3619.2329302893236  ; probF:  -3619.23179559942 xxxx:  0.00113468990366527